In [1]:
from os import getcwd
from os.path import join, basename
from sys import path 

libs_dir = join("/".join(getcwd().split("/")[:-1]))
path.append(libs_dir)

filename = basename(globals()['__vsc_ipynb_file__']).split(".")[0]

import numpy as np
from libs.qchannel_model import *
from libs.figure_config import *
from libs.default_parameters import *
from scipy.special import gamma
import mpmath as mp

In [2]:
sigma_theta_x = theta_rad/8
sigma_theta_y = theta_rad/8
ra = 0.75

In [3]:
tau_zen = 0.91
zenith_angle_deg = 0
zenith_angle_rad = np.radians(zenith_angle_deg)

slant_distance = compute_slant_distance(h_s, h_OGS, zenith_angle_rad)
w_L = slant_distance * theta_rad

sigma_x = sigma_theta_x * slant_distance
sigma_y = sigma_theta_y * slant_distance

w_Leq_squared = equivalent_beam_width_squared(ra, w_L)
w_Leq = np.sqrt(w_Leq_squared)
sigma_R_squared = rytov_variance(
        wavelength, zenith_angle_rad, h_OGS, h_atm, Cn2_profile)
sigma_mod = compute_sigma_mod(mu_x, mu_y, sigma_x, sigma_y)
varphi_mod = sigma_to_variance(sigma_mod, w_Leq)
A_mod = mod_jitter(mu_x, mu_y, sigma_x, sigma_y, w_L, w_Leq, ra)
mu = (sigma_R_squared/2) * (1 + 2 * varphi_mod**2)

eta_l = compute_atm_loss(tau_zen, zenith_angle_rad)

In [4]:
def f(eta, k):
    a = eta**(varphi_mod**2-1+k)
    b = erfc(
        (np.log(eta / (A_mod * eta_l)) + mu)
        / (np.sqrt(2) * np.sqrt(sigma_R_squared))
        )
    return a * b

In [5]:
def f_test(eta, k):
    term_1 = np.exp(np.sqrt(2*sigma_R_squared) * (varphi_mod**2+k) * eta)
    term_2 = erfc(
        eta
    )

    return term_1 * term_2

In [6]:
avg_yield_test, _ = quad(f_test, -2, 20, args=(5))
print(avg_yield_test)

3.624335189189155e+66


In [7]:
def f_closed_form(k):
    term_1 = (
        np.sqrt(2) * np.sqrt(sigma_R_squared)
        * (
            A_mod * eta_l * np.exp(-mu)
        )**(varphi_mod**2+k)
    )

    z1 = -np.sqrt(2*sigma_R_squared) * (varphi_mod**2+k)
    term_21 = (
        gamma(1)
        * float(mp.hyp2f2(1, 0.5, 0.5, 1.5, z1**2/4))
    )
    term_22 = (
        (z1/2)
        * gamma(1)
        * float(mp.hyp2f2(1, 1.5, 1.5, 2, z1**2/4))
    )
    res1 = term_1 * (term_21 - term_22)/np.sqrt(np.pi)

    z2 = np.sqrt(2*sigma_R_squared) * (varphi_mod**2+k)
    term_31 = (
        gamma(1)
        * float(mp.hyp2f2(1, 0.5, 0.5, 1.5, z2**2/4))
    )
    term_32 = (
        (z2/2)
        * gamma(1)
        * float(mp.hyp2f2(1, 1.5, 1.5, 2, z2**2/4))
    )
    res2 = term_1 * (term_31 - term_32)/np.sqrt(np.pi)
    # res3 = term_1 * 2 * term_21/np.sqrt(np.pi)
    # return res3
    return res1 + res2

def f_closed_form_short(k):
    term_1 = (
        np.sqrt(2) * np.sqrt(sigma_R_squared)
        * (
            A_mod * eta_l * np.exp(-mu)
        )**(varphi_mod**2+k)
    )

    z1 = -np.sqrt(2*sigma_R_squared) * (varphi_mod**2+k)
    term_21 = float(mp.hyp2f2(1, 0.5, 0.5, 1.5, z1**2/4))
    res3 = term_1 * 2 * term_21/np.sqrt(np.pi)
    return res3

In [8]:
print(f_closed_form_short(1))
print(f_closed_form(1))

4.041198579155187e-108
4.041198579155187e-108


In [23]:
print(f_closed_form(10))
avg_yield, _ = quad(f, 0, np.infty, args=(10))
print(avg_yield)

9.340284335940118e-118
9.340202768829622e-118


# Next step

In [10]:
def f2(eta):
    c = np.exp(-n_s*eta)
    a = eta**(varphi_mod**2-1)
    b = erfc(
        (np.log(eta / (A_mod * eta_l)) + mu)
        / (np.sqrt(2) * np.sqrt(sigma_R_squared))
        )
    return a * b * c

In [21]:
avg_yield_2, _ = quad(f2, 0, 1)
print(avg_yield_2)

1.2807052075130401e-106


In [12]:
def f2_closed_form(iters):
    res = 0
    for idx in range(iters):
        term_1 = (-n_s)**idx/np.math.factorial(idx)
        term_2 = f_closed_form(idx)
        res += term_1 + term_2
        # print(term_1, term_2)
    return res

In [13]:
print(f2_closed_form(20))

0.6065306597126333


# Final step

In [14]:
def f3(eta):
    a = transmitivity_pdf(
        eta, mu_x, mu_y, sigma_x, sigma_y, zenith_angle_rad,
        w_L, w_Leq, tau_zen, varphi_mod, wavelength, h_OGS,
        h_atm, Cn2_profile, ra)
    b = e_0 * (p_dark*(1+p_AP)) + (e_pol+e_0*p_AP) * (1-np.exp(-n_s*eta))

    return a * b

avg_yield_3, _ = quad(f3, 0, np.infty)
print(avg_yield_3)

0.0003603777926624077


In [15]:
def f3_closed_form(iters):
    term_1 = e_0 * (p_dark*(1+p_AP)) + e_pol + e_0*p_AP
    term_2 = (
        (e_pol + e_0*p_AP)
        * ((varphi_mod**2) / (2 * (A_mod * eta_l)**(varphi_mod**2)))
        * np.exp(
        (sigma_R_squared / 2) * varphi_mod**2 * (1 + varphi_mod**2)
        )
    )
    term_3 = f2_closed_form(iters)
    # print(term_1, term_2, term_3)
    res = term_1 - term_2 * term_3  
    return res

In [16]:
print(f3_closed_form(50))

-9.37545673015224e+103


# F4

In [17]:
def f4(eta):
    a = transmitivity_pdf(
        eta, mu_x, mu_y, sigma_x, sigma_y, zenith_angle_rad,
        w_L, w_Leq, tau_zen, varphi_mod, wavelength, h_OGS,
        h_atm, Cn2_profile, ra)
    b = np.exp(-n_s*eta)

    return a * b

avg_yield_4, _ = quad(f4, 0, np.infty)
print(avg_yield_4)

0.984531110366942


In [18]:
def f4_closed_form(iters):
    term_1 = (
        ((varphi_mod**2) / (2 * (A_mod * eta_l)**(varphi_mod**2)))
        * np.exp(
        (sigma_R_squared / 2) * varphi_mod**2 * (1 + varphi_mod**2)
        )
    )
    term_2 = f2_closed_form(iters)
    # term_2, _ = quad(f2, 0, 1)
    res = term_1 * term_2 
    return res

In [19]:
f4_closed_form(20)

4.68772836507612e+105

In [20]:
# def f32(eta):
#     term_1 = transmitivity_pdf(
#         eta, mu_x, mu_y, sigma_x, sigma_y, zenith_angle_rad,
#         w_L, w_Leq, tau_zen, varphi_mod, wavelength, h_OGS,
#         h_atm, Cn2_profile, ra)
#     term_2 = (e_pol+e_0*p_AP) * np.exp(-n_s*eta)
#     return term_1 * term_2

# avg_yield_4, _ = quad(f32, 0, np.inf)
# print(e_0 * (p_dark*(1+p_AP)) + e_pol + e_0*p_AP - avg_yield_4)
# print(avg_yield_4)